In [6]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Define paths
data_dir = '/kaggle/input/rice-plant-dataset/Rice Seed Dataset'
healthy_dir = os.path.join(data_dir, 'Healthy_Images')
unhealthy_dir = os.path.join(data_dir, 'Unhealthy_Images')

# List all image files
healthy_files = [os.path.join(healthy_dir, f) for f in os.listdir(healthy_dir) if f.endswith('.jpg')]
unhealthy_files = [os.path.join(unhealthy_dir, f) for f in os.listdir(unhealthy_dir) if f.endswith('.jpg')]

# Create labels
healthy_labels = [0] * len(healthy_files)
unhealthy_labels = [1] * len(unhealthy_files)

# Combine files and labels
files = healthy_files + unhealthy_files
labels = healthy_labels + unhealthy_labels

# Split data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, random_state=42)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self.files = files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create custom datasets
train_dataset = CustomDataset(train_files, train_labels, transform=transform)
val_dataset = CustomDataset(val_files, val_labels, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load AlexNet model
model = models.alexnet(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # 2 classes: healthy and unhealthy

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print training loss
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print validation accuracy
    print(f'Validation Accuracy: {correct/total:.4f}')

print('Training complete!')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 124MB/s]  


Epoch 1/5, Loss: 0.9127
Validation Accuracy: 0.5000
Epoch 2/5, Loss: 0.7000
Validation Accuracy: 0.5000
Epoch 3/5, Loss: 0.6967
Validation Accuracy: 0.5000
Epoch 4/5, Loss: 0.6927
Validation Accuracy: 0.5000
Epoch 5/5, Loss: 0.6949
Validation Accuracy: 0.5000
Training complete!


In [15]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on validation set: {accuracy:.4f}')


Accuracy on validation set: 0.5000


In [18]:
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return predicted.item()

# Example usage
image_path = '/kaggle/input/rice-plant-dataset/Rice Seed Dataset/Healthy_Images/healthy (1).jpg'
prediction = predict_image(image_path)
if prediction == 0:
    print('The rice plant is healthy.')
else:
    print('The rice plant is unhealthy.')


The rice plant is healthy.
